In [1]:
import warnings
warnings.filterwarnings("ignore")
from env import host, user, password
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text, export_graphviz
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import os
import acquire
import prepare

In [2]:
telco_db = acquire.get_telco_db('telco_churn')

In [3]:
telco_db.head()

,payment_type_id,contract_type_id,internet_service_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,internet_service_type,contract_type,payment_type
0,2,1,3,0030-FNXPP,Female,0,No,No,3,Yes,...,No internet service,No internet service,No internet service,No,19.85,57.2,No,None,Month-to-month,Mailed check
1,2,1,3,0031-PVLZI,Female,0,Yes,Yes,4,Yes,...,No internet service,No internet service,No internet service,No,20.35,76.35,Yes,None,Month-to-month,Mailed check
2,1,1,3,0098-BOWSO,Male,0,No,No,27,Yes,...,No internet service,No internet service,No internet service,Yes,19.40,529.8,No,None,Month-to-month,Electronic check
3,1,1,3,0107-WESLM,Male,0,No,No,1,Yes,...,No internet service,No internet service,No internet service,Yes,19.85,19.85,Yes,None,Month-to-month,Electronic check
4,3,1,3,0114-RSRRW,Female,0,Yes,No,10,Yes,...,No internet service,No internet service,No internet service,Yes,19.95,187.75,No,None,Month-to-month,Bank transfer (automatic)


In [4]:
telco_db.columns

Index(['payment_type_id', 'contract_type_id', 'internet_service_type_id',
       'customer_id', 'gender', 'senior_citizen', 'partner', 'dependents',
       'tenure', 'phone_service', 'multiple_lines', 'online_security',
       'online_backup', 'device_protection', 'tech_support', 'streaming_tv',
       'streaming_movies', 'paperless_billing', 'monthly_charges',
       'total_charges', 'churn', 'internet_service_type', 'contract_type',
       'payment_type'],
      dtype='object')

In [5]:
telco_db[telco_db['total_charges'].str.contains(" ")]

,payment_type_id,contract_type_id,internet_service_type_id,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,...,tech_support,streaming_tv,streaming_movies,paperless_billing,monthly_charges,total_charges,churn,internet_service_type,contract_type,payment_type
3977,2,2,3,2923-ARZLG,Male,0,Yes,Yes,0,Yes,...,No internet service,No internet service,No internet service,Yes,19.70,,No,None,One year,Mailed check
5512,2,3,3,2520-SGTTA,Female,0,Yes,Yes,0,Yes,...,No internet service,No internet service,No internet service,No,20.00,,No,None,Two year,Mailed check
5548,2,3,3,3115-CZMZD,Male,0,No,Yes,0,Yes,...,No internet service,No internet service,No internet service,No,20.25,,No,None,Two year,Mailed check
5558,2,3,3,3213-VVOLG,Male,0,Yes,Yes,0,Yes,...,No internet service,No internet service,No internet service,No,25.35,,No,None,Two year,Mailed check
5629,2,3,3,4367-NUYAO,Male,0,Yes,Yes,0,Yes,...,No internet service,No internet service,No internet service,No,25.75,,No,None,Two year,Mailed check
5838,2,3,3,7644-OMVMY,Male,0,Yes,Yes,0,Yes,...,No internet service,No internet service,No internet service,No,19.85,,No,None,Two year,Mailed check
6500,4,3,1,1371-DWPAZ,Female,0,Yes,Yes,0,No,...,Yes,Yes,No,No,56.05,,No,DSL,Two year,Credit card (automatic)
6571,3,3,1,2775-SEFEE,Male,0,No,Yes,0,Yes,...,Yes,No,No,Yes,61.90,,No,DSL,Two year,Bank transfer (automatic)
6651,2,3,1,4075-WKNIU,Female,0,Yes,Yes,0,Yes,...,Yes,Yes,No,No,73.35,,No,DSL,Two year,Mailed check
6670,3,3,1,4472-LVYGI,Female,0,Yes,Yes,0,No,...,Yes,Yes,No,Yes,52.55,,No,DSL,Two year,Bank transfer (automatic)


In [6]:
telco_db['total_charges'].value_counts()

           11
20.2       11
19.75       9
20.05       8
19.9        8
           ..
224.85      1
72.4        1
155.65      1
2651.1      1
7382.85     1
Name: total_charges, Length: 6531, dtype: int64

Need Dummy columns for: multiple_lines, online_security, internet_service_type, contract_type, payment_type

In [7]:
def prep_telco(df):
    
    df = telco_db.drop(columns = 'payment_type_id',inplace = True)
    df = telco_db.drop(columns = 'contract_type_id',inplace = True)
    df = telco_db.drop(columns = 'internet_service_type_id',inplace = True)
    df = telco_db.drop(columns = 'customer_id',inplace = True)
    df = telco_db.drop(columns = 'online_backup',inplace = True)
    df = telco_db.drop(columns = 'device_protection',inplace = True)
    df = telco_db.drop(columns = 'tech_support',inplace = True)
    df = telco_db.drop(columns = 'streaming_tv',inplace = True)
    df = telco_db.drop(columns = 'streaming_movies',inplace = True)
    df = telco_db.replace({'gender':{'Male':0,'Female':1}},inplace = True)
    df = telco_db.replace({'partner':{'No':0,'Yes':1}},inplace = True)
    df = telco_db.replace({'dependents':{'No':0,'Yes':1}},inplace = True)
    df = telco_db.replace({'phone_service':{'No':0,'Yes':1}},inplace = True)
    df = telco_db.replace({'paperless_billing':{'No':0,'Yes':1}},inplace = True)
    df = telco_db.replace({'churn':{'No':0,'Yes':1}},inplace = True)
    df = telco_db.rename(columns = {'gender':'gender_is_female'},inplace = True)
    df = telco_db.drop(telco_db[telco_db['total_charges'].str.contains(" ")].index, inplace = True)


    return df

In [8]:
prep_telco(telco_db)

In [9]:
df = telco_db
df.head()

,gender_is_female,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,online_security,paperless_billing,monthly_charges,total_charges,churn,internet_service_type,contract_type,payment_type
0,1,0,0,0,3,1,No,No internet service,0,19.85,57.2,0,None,Month-to-month,Mailed check
1,1,0,1,1,4,1,No,No internet service,0,20.35,76.35,1,None,Month-to-month,Mailed check
2,0,0,0,0,27,1,No,No internet service,1,19.40,529.8,0,None,Month-to-month,Electronic check
3,0,0,0,0,1,1,No,No internet service,1,19.85,19.85,1,None,Month-to-month,Electronic check
4,1,0,1,0,10,1,No,No internet service,1,19.95,187.75,0,None,Month-to-month,Bank transfer (automatic)


In [10]:
dummy_df = pd.get_dummies(df[['multiple_lines','online_security','internet_service_type','contract_type','payment_type']], dummy_na=False, drop_first=False)
dummy_df.head()

,multiple_lines_No,multiple_lines_No phone service,multiple_lines_Yes,online_security_No,online_security_No internet service,online_security_Yes,internet_service_type_DSL,internet_service_type_Fiber optic,internet_service_type_None,contract_type_Month-to-month,contract_type_One year,contract_type_Two year,payment_type_Bank transfer (automatic),payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1
1,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,1
2,1,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0
3,1,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0
4,1,0,0,0,1,0,0,0,1,1,0,0,1,0,0,0


In [11]:
df = pd.concat([df, dummy_df], axis=1)
df = df.drop(columns = ['multiple_lines','online_security','internet_service_type','contract_type','payment_type'])
df.head()

,gender_is_female,senior_citizen,partner,dependents,tenure,phone_service,paperless_billing,monthly_charges,total_charges,churn,...,internet_service_type_DSL,internet_service_type_Fiber optic,internet_service_type_None,contract_type_Month-to-month,contract_type_One year,contract_type_Two year,payment_type_Bank transfer (automatic),payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
0,1,0,0,0,3,1,0,19.85,57.2,0,...,0,0,1,1,0,0,0,0,0,1
1,1,0,1,1,4,1,0,20.35,76.35,1,...,0,0,1,1,0,0,0,0,0,1
2,0,0,0,0,27,1,1,19.40,529.8,0,...,0,0,1,1,0,0,0,0,1,0
3,0,0,0,0,1,1,1,19.85,19.85,1,...,0,0,1,1,0,0,0,0,1,0
4,1,0,1,0,10,1,1,19.95,187.75,0,...,0,0,1,1,0,0,1,0,0,0


In [12]:
df.columns

Index(['gender_is_female', 'senior_citizen', 'partner', 'dependents', 'tenure',
       'phone_service', 'paperless_billing', 'monthly_charges',
       'total_charges', 'churn', 'multiple_lines_No',
       'multiple_lines_No phone service', 'multiple_lines_Yes',
       'online_security_No', 'online_security_No internet service',
       'online_security_Yes', 'internet_service_type_DSL',
       'internet_service_type_Fiber optic', 'internet_service_type_None',
       'contract_type_Month-to-month', 'contract_type_One year',
       'contract_type_Two year', 'payment_type_Bank transfer (automatic)',
       'payment_type_Credit card (automatic)', 'payment_type_Electronic check',
       'payment_type_Mailed check'],
      dtype='object')

In [13]:
churned = df[df.churn == 1]
churned.head()

,gender_is_female,senior_citizen,partner,dependents,tenure,phone_service,paperless_billing,monthly_charges,total_charges,churn,...,internet_service_type_DSL,internet_service_type_Fiber optic,internet_service_type_None,contract_type_Month-to-month,contract_type_One year,contract_type_Two year,payment_type_Bank transfer (automatic),payment_type_Credit card (automatic),payment_type_Electronic check,payment_type_Mailed check
1,1,0,1,1,4,1,0,20.35,76.35,1,...,0,0,1,1,0,0,0,0,0,1
3,0,0,0,0,1,1,1,19.85,19.85,1,...,0,0,1,1,0,0,0,0,1,0
5,1,0,0,0,3,1,0,19.85,63.75,1,...,0,0,1,1,0,0,0,0,0,1
11,1,0,0,0,3,1,0,19.75,58.85,1,...,0,0,1,1,0,0,0,0,1,0
12,0,0,0,0,5,1,0,21.05,113.85,1,...,0,0,1,1,0,0,0,0,0,1


In [14]:
# Split Data
train_validate, test = train_test_split(df, test_size=.2, 
                                        random_state=123, 
                                        stratify=df.churn)
train, validate = train_test_split(train_validate, test_size=.3, 
                                   random_state=123, 
                                   stratify=train_validate.churn)

In [15]:
X_train = train.drop(columns=['churn'])
y_train = train.churn

X_validate = validate.drop(columns=['churn'])
y_validate = validate.churn

X_test = test.drop(columns=['churn'])
y_test = test.churn


In [16]:
logit = LogisticRegression(C=1)

In [17]:
logit.fit(X_train, y_train)

LogisticRegression(C=1)

In [18]:
print('Coefficient: \n', logit.coef_)
print('Intercept: \n', logit.intercept_)

Coefficient: 
 [[-7.77859226e-02  3.55044103e-01  1.81753987e-01 -3.30163789e-01
  -5.30194372e-02 -3.90543267e-01  3.66420066e-01  5.87638470e-03
   1.79478709e-04 -3.23028577e-01  4.37329792e-02 -6.75146901e-02
   1.81137836e-01 -3.71153212e-01 -1.56794913e-01 -3.66873911e-01
   3.91216835e-01 -3.71153212e-01  4.34119541e-01 -2.21596355e-01
  -5.59333475e-01 -1.42407138e-01 -2.05342715e-01  1.78457943e-01
  -1.77518378e-01]]
Intercept: 
 [-0.35144841]


In [19]:
y_pred = logit.predict(X_train)


In [20]:
y_pred_proba = logit.predict_proba(X_train)


In [21]:
print('Accuracy of Logistic Regression classifier on training set: {:.2f}'
     .format(logit.score(X_train, y_train)))

Accuracy of Logistic Regression classifier on training set: 0.81


In [22]:
train.columns

Index(['gender_is_female', 'senior_citizen', 'partner', 'dependents', 'tenure',
       'phone_service', 'paperless_billing', 'monthly_charges',
       'total_charges', 'churn', 'multiple_lines_No',
       'multiple_lines_No phone service', 'multiple_lines_Yes',
       'online_security_No', 'online_security_No internet service',
       'online_security_Yes', 'internet_service_type_DSL',
       'internet_service_type_Fiber optic', 'internet_service_type_None',
       'contract_type_Month-to-month', 'contract_type_One year',
       'contract_type_Two year', 'payment_type_Bank transfer (automatic)',
       'payment_type_Credit card (automatic)', 'payment_type_Electronic check',
       'payment_type_Mailed check'],
      dtype='object')

In [23]:
fiber = telco_db[telco_db['internet_service_type']=='Fiber optic']
fiber.multiple_lines.value_counts()

Yes    1938
No     1158
Name: multiple_lines, dtype: int64